In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_excel('Input.xlsx')
df.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [ ]:
def get_title(soup):
  title = soup.find("h1", attrs={"class":'entry-title'})
  if title !=None:
    title_string = title.text.strip()
    return title_string
  else:
      title = soup.find("h1", attrs={"class":'tdb-title-text'})
      if title !=None:
        title_string = title.text.strip()
        return title_string
      else:
        return np.nan

def get_article(soup):
  article = soup.find("div", attrs={"class":'td-post-content tagdiv-type'})
  if article !=None: 
    article.find_all("p")
    article_string = ''.join([paragraph.get_text().strip() for paragraph in article])
    return article_string
  else:
    article = soup.find("div", attrs={"class":'td_block_wrap tdb_single_content tdi_130 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'})
    if article !=None: 
      article=article.find("div",attrs={"class":'tdb-block-inner td-fix-index'})
      if article !=None: 
        article=article.find_all("p")
        article_string = ''.join([paragraph.get_text().strip() for paragraph in article])
        return article_string

    else:
      return np.nan

In [ ]:
d = {"title":[], "articles":[]}
for link in df['URL']:
        webpage = requests.get(link)

        soup = BeautifulSoup(webpage.content, "html.parser")

        d['title'].append(get_title(soup))
        d['articles'].append(get_article(soup))

In [ ]:
df['title']=d['title']
df['articles']=d['articles']

In [ ]:
df.head()

,URL_ID,URL,title,articles
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction“If anything kills over 10 million...
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ..."
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,IntroductionAI is rapidly evolving in the empl...
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...


In [ ]:
df.isnull().sum()

URL_ID      0
URL         0
title       3
articles    3
dtype: int64

In [ ]:
len(df.articles[0])

12022

In [ ]:
drop_index=df.articles[df.articles.isnull()].index
drop_index

Int64Index([7, 20, 107], dtype='int64')

In [ ]:
df.dropna(inplace=True) # drop the values where page was not on particular url
df.isnull().sum()

URL_ID      0
URL         0
title       0
articles    0
dtype: int64

In [ ]:
df.head()

,URL_ID,URL,title,articles
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction“If anything kills over 10 million...
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ..."
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,IntroductionAI is rapidly evolving in the empl...
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...


In [ ]:
import re
def clean_data(text):
  text = re.sub(r'\s+', ' ', text)
  text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  return text

df.articles=df.articles.apply(clean_data)

In [ ]:
df.head()

,URL_ID,URL,title,articles
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,IntroductionIf anything kills over 10 million ...
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,Human minds a fascination in itself carrying t...
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,IntroductionAI is rapidly evolving in the empl...
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,Anything that could give rise to smarterthanhu...
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,Machine intelligence is the last invention tha...


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import glob
import os

def get_words_from_directory(directory):
    words = []
    file_paths = glob.glob(os.path.join(directory, '*.txt'))  # Get all text files in the directory
    
    for file_path in file_paths:
        with open(file_path, 'r',encoding='utf-8', errors='ignore') as f:
            content = f.read()
            # Tokenize the content into words
            content_words = content.split()
            words.extend(content_words)
    

    return words

stop_words=get_words_from_directory('/content/stopwords/')

In [ ]:
def clean_articles(text):
    tokens = nltk.word_tokenize(text)

    # Get the list of stopwords
    stopwords_list = set(stop_words)
    lemmatizer = WordNetLemmatizer()
    # Remove stopwords from the tokens
    filtered_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stopwords_list]
    cleaned_text_without_stopwords = " ".join(filtered_tokens)
    return cleaned_text_without_stopwords

df.articles=df.articles.apply(clean_articles)

In [ ]:
output_df= pd.read_excel('Output Data Structure.xlsx')
output_df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38,https://insights.blackcoffer.com/what-if-the-c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39,https://insights.blackcoffer.com/what-jobs-wil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40,https://insights.blackcoffer.com/will-machine-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41,https://insights.blackcoffer.com/will-ai-repla...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
output_df=output_df.drop(drop_index,axis=0)

In [ ]:
def create_word_dictionary(positive_file, negative_file):
    word_dictionary = {
        'positive': set(),
        'negative': set()
    }

    with open(positive_file, 'r',encoding='utf-8',errors='ignore') as file:
        positive_words = file.read().split()
        word_dictionary['positive'].update(positive_words)

    with open(negative_file, 'r',encoding='utf-8',errors='ignore') as file:
        negative_words = file.read().split()
        word_dictionary['negative'].update(negative_words)

    return word_dictionary
 
dict_pos_neg = create_word_dictionary('positive-words.txt', 'negative-words.txt')

dict_pos_neg['negative']

{'jutter',
 'snappishly',
 'ruinous',
 'ineffectually',
 'aggravate',
 'disrespectful',
 'rejection',
 'viciously',
 'spoil',
 'onerously',
 'hegemonistic',
 'wariness',
 'forlorn',
 'rattle',
 'provoke',
 'confusions',
 'spoonfed',
 'bitch',
 'disillusionment',
 'rascals',
 'fidget',
 'distorted',
 'unaccustomed',
 'frenetic',
 'unsavory',
 'incense',
 'corrupting',
 'molest',
 'strained',
 'inelegant',
 'stains',
 'wack',
 'distrustful',
 'niggle',
 'discouragingly',
 'gloomy',
 'peeve',
 'slow-moving',
 'uncivil',
 'rremediable',
 'slumpping',
 'busybody',
 'nervous',
 'torturing',
 'fatique',
 'indolent',
 'infected',
 'inflationary',
 'caustic',
 'judder',
 'vulnerable',
 'stumps',
 'suffered',
 'flake',
 'sulk',
 'ominous',
 'stingy',
 'naughty',
 'accusing',
 'untested',
 'destructive',
 'swipe',
 'despondence',
 'gritty',
 'virulence',
 'weirdly',
 'discontentedly',
 'vexing',
 'spewed',
 'erase',
 'blockage',
 'inarticulate',
 'precariously',
 'manipulation',
 'catastrophies',

In [ ]:
def calculate_positive_score(text, positive_words):
    words = nltk.word_tokenize(text)
    positive_score = sum(1 for word in words if word in positive_words)
    return positive_score

def calculate_negative_score(text, negative_words):
    words = text.split()
    negative_score = sum(1 for word in words if word in negative_words)
    return negative_score

output_df['POSITIVE SCORE']=df.articles.apply(lambda x: calculate_positive_score(x,dict_pos_neg['positive']))
output_df['NEGATIVE SCORE']=df.articles.apply(lambda x: calculate_negative_score(x,dict_pos_neg['negative']))


In [ ]:
def calculate_polarity_score(text, positive_words, negative_words):
    words =nltk.word_tokenize(text)
    positive_score = sum(1 for word in words if word in positive_words)
    negative_score = sum(1 for word in words if word in negative_words)
    
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    return polarity_score

output_df['POLARITY SCORE']=df.articles.apply(lambda x: calculate_polarity_score(x,dict_pos_neg['positive'],dict_pos_neg['negative']))

In [ ]:
def calculate_subjectivity_score(text, positive_words, negative_words):
    words = nltk.word_tokenize(text)
    positive_score = sum(1 for word in words if word in positive_words)
    negative_score = sum(1 for word in words if word in negative_words)
    
    subjectivity_score = (positive_score + negative_score) / (len(words) + 0.000001)
    return subjectivity_score

output_df['SUBJECTIVITY SCORE']=df.articles.apply(lambda x: calculate_subjectivity_score(x,dict_pos_neg['positive'],dict_pos_neg['negative']))

In [ ]:
def average_sen_length(text):
    sents = nltk.sent_tokenize(text)
    
    total_words = 0
    total_sents = len(sents)
    
    for sentence in sents:
        words = nltk.word_tokenize(sentence)
        total_words += len(words)
    
    if total_sents > 0:
        average_length = total_words / total_sents
        return average_length
    else:
        return 0

output_df['AVG SENTENCE LENGTH']=df['articles'].apply(average_sen_length)

In [ ]:
def syllable_counts(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count += 1
    if count == 0:
        count += 1
    return count

def complex_word_percentage(text):

    words = nltk.word_tokenize(text)
    
    complex_word_count = 0
    total_word_count = len(words)
    
    for word in words:
        syllable_count = syllable_counts(word)
        if syllable_count > 2:
            complex_word_count += 1
    
    if total_word_count > 0:
        complex_word_percentage = (complex_word_count / total_word_count) * 100
        return complex_word_percentage
    else:
        return 0

output_df['PERCENTAGE OF COMPLEX WORDS']=df['articles'].apply(complex_word_percentage)

In [ ]:
def fog_index(text):
    
    sentences = nltk.sent_tokenize(text)
    total_words = 0
    total_sentences = len(sentences)
    complex_word_count = 0
 
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        total_words += len(words)
        for word in words:
            syllable_count = syllable_counts(word)
            if syllable_count > 2:
                complex_word_count += 1
    
    average_sentence_length = total_words / total_sentences

    complex_word_percentage = (complex_word_count / total_words) * 100

    fog_index = 0.4 * (average_sentence_length + complex_word_percentage)
    
    return fog_index


output_df['FOG INDEX']=df['articles'].apply(fog_index)

In [ ]:
def calculate_average_words_per_sentence(text):
    sentences = nltk.sent_tokenize(text)
    
    total_words = 0
    total_sentences = len(sentences)
    
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        total_words += len(words)
    
    if total_sentences > 0:
        average_words_per_sentence = total_words / total_sentences
        return average_words_per_sentence
    else:
        return 0

output_df['AVG NUMBER OF WORDS PER SENTENCE']=df['articles'].apply(calculate_average_words_per_sentence)

In [ ]:
def calculate_complex_word_count(text, syllable_threshold=2):
    words = nltk.word_tokenize(text)
    
    complex_word_count = 0
    
    for word in words:
        syllable_count = syllable_counts(word)
        if syllable_count > syllable_threshold:
            complex_word_count += 1
    
    return complex_word_count

output_df['COMPLEX WORD COUNT']=df['articles'].apply(calculate_complex_word_count)

In [ ]:
output_df['WORD COUNT']=df['articles'].apply(lambda x: len(nltk.word_tokenize(x)) )

In [ ]:
def calculate_syllables_per_word(text):
    words = nltk.word_tokenize(text)
    
    total_syllables = 0
    total_words = len(words)
    
    for word in words:
        syllable_count = syllable_counts(word)
        total_syllables += syllable_count
    
    if total_words > 0:
        syllables_per_word = total_syllables / total_words
        return syllables_per_word
    else:
        return 0

output_df['SYLLABLE PER WORD']=df['articles'].apply(calculate_syllables_per_word)

In [ ]:
def calculate_personal_pronouns(text):
    words = nltk.word_tokenize(text)
    
    personal_pronoun_count = 0
    personal_pronouns=["i", "me", "my", "mine", "myself","you", "your", "yours", "yourself","he", "him", "his", "himself","she", "her", "hers", "herself","it", "its", "itself","we", "us", "our", "ours", "ourselves","they", "them", "their", "theirs", "themselves"]
    for word in words:
        word_lower = word.lower()
        if word_lower in personal_pronouns:
            personal_pronoun_count += 1
    
    return personal_pronoun_count

output_df['PERSONAL PRONOUNS']=df['articles'].apply(calculate_personal_pronouns)

In [ ]:
def calculate_average_word_length(text):
    words = nltk.word_tokenize(text)
    
    total_characters = 0
    
    for word in words:
        total_characters += len(word)
    
    if len(words) > 0:
        average_word_length = total_characters / len(words)
        return average_word_length
    else:
        return 0

output_df['AVG WORD LENGTH']=df['articles'].apply(calculate_average_word_length)

In [ ]:
output_df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,88,32,0.466667,0.068847,1743.0,24.956971,707.182788,1743.0,435,1743,1.915663,29,5.620769
1,38,https://insights.blackcoffer.com/what-if-the-c...,83,36,0.394958,0.085366,1394.0,16.642755,564.257102,1394.0,232,1394,1.621234,61,4.761836
2,39,https://insights.blackcoffer.com/what-jobs-wil...,76,41,0.299145,0.069934,1673.0,24.387328,678.954931,1673.0,408,1673,1.865511,36,5.378362
3,40,https://insights.blackcoffer.com/will-machine-...,83,25,0.537037,0.066873,1615.0,16.718266,652.687307,1615.0,270,1615,1.647678,61,4.803096
4,41,https://insights.blackcoffer.com/will-ai-repla...,71,28,0.434343,0.058030,1706.0,19.577960,690.231184,1706.0,334,1706,1.715709,59,5.067409


In [ ]:
output_df.isnull().sum()

URL_ID                              0
URL                                 0
POSITIVE SCORE                      0
NEGATIVE SCORE                      0
POLARITY SCORE                      0
SUBJECTIVITY SCORE                  0
AVG SENTENCE LENGTH                 0
PERCENTAGE OF COMPLEX WORDS         0
FOG INDEX                           0
AVG NUMBER OF WORDS PER SENTENCE    0
COMPLEX WORD COUNT                  0
WORD COUNT                          0
SYLLABLE PER WORD                   0
PERSONAL PRONOUNS                   0
AVG WORD LENGTH                     0
dtype: int64

In [ ]:
output_df.to_excel('output_data.xlsx', index=False)